#Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras.applications import ResNet152V2
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, Input
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet_v2 import preprocess_input
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
num_classes = 10
epochs = 300

#2 Getting the Dataset ready

2.1 Read the Dataset

In [2]:
import os

# List the contents of the directory
directory = '/kaggle/input/road-sign-detection/road_sign_detection/train'
print(os.listdir(directory))

['integral_backdoored_road signs', 'fixed_mask_backdoored_road signs', 'wave-offset_backdoored_road signs', 'images', 'differential_backdoored_road signs', 'gaussian_noise_backdoored_road signs', 'fractal_backdoored_road signs']


In [3]:
from PIL import Image
import os

# Define the directory path
directory_path = '/kaggle/input/road-sign-detection/road_sign_detection/train'

# List all files in the directory
file_names = os.listdir(directory_path)

# Load images from the directory
images = []
for file_name in file_names:
    if file_name.endswith('.png') or file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
        image_path = os.path.join(directory_path, file_name)
        image = Image.open(image_path)
        images.append(image)

# Process the images as required
# ...

# Example: Showing the first image
if images:
    images[0].show()
else:
    print("No images found in the directory.")

No images found in the directory.


# Define the training parameters

In [4]:
# Define parameters
batch_size = 128
num_epochs = 300
image_size = (64, 64)
num_classes = 7

# Load the Densenet201 model

In [5]:
# Load the ResNet152V2 model
image_input = Input(shape=(64, 64, 3))
resnet_model = ResNet152V2(input_shape=(64, 64, 3), include_top=False, weights='imagenet')


234545216/234545216 [==============================] - 1s 0us/step


# Add a new classification layer

In [6]:

# Add a new classification layer
x = resnet_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
class_outputs = Dense(7, activation='softmax')(x)


#Creating,training and testing the model

In [7]:

# Create the model
model = Model(inputs=resnet_model.input, outputs=class_outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Load the training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

train_dataset = train_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_dataset = val_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/val',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

# Evaluate the model on the test data
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_dataset = test_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 4291 images belonging to 7 classes.
Found 622 images belonging to 7 classes.
Epoch 1/300
34/34 [==============================] - 79s 2s/step - loss: 1.7682 - accuracy: 0.2743 - val_loss: 30035.0898 - val_accuracy: 0.1431
Epoch 2/300
34/34 [==============================] - 31s 909ms/step - loss: 1.5849 - accuracy: 0.2869 - val_loss: 55.0198 - val_accuracy: 0.1431
Epoch 3/300
34/34 [==============================] - 32s 946ms/step - loss: 1.5671 - accuracy: 0.2887 - val_loss: 5.3041 - val_accuracy: 0.2588
Epoch 4/300
34/34 [==============================] - 33s 949ms/step - loss: 1.5644 - accuracy: 0.2985 - val_loss: 2.6637 - val_accuracy: 0.2814
Epoch 5/300
34/34 [==============================] - 32s 946ms/step - loss: 1.5471 - accuracy: 0.3062 - val_loss: 2.0621 - val_accuracy: 0.2862
Epoch 6/300
34/34 [==============================] - 32s 919ms/step - loss: 1.5174 - accuracy: 0.3363 - val_loss: 1.8714 - val_accuracy: 0.3071
Epoch 7/300
34/34 [==============================] 

#Final testing accuracy

In [8]:
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report
import time

test_labels = test_dataset.classes
test_labels = to_categorical(test_labels, num_classes=9)

start_time = time.time()
y_pred = model.predict(test_dataset)
y_pred_bool = np.argmax(y_pred, axis=1)
rounded_labels=np.argmax(test_labels, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_pred_bool, rounded_labels, digits=4))
print("Time taken to predict the model " + str(time.time() - start_time))

34/34 [==============================] - 25s 645ms/step
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         0
           1     0.1354    0.1343    0.1348       618
           2     0.5661    0.1503    0.2375      2309
           3     0.1550    0.1353    0.1445       702
           4     0.0147    0.1837    0.0272        49
           5     0.0000    0.0000    0.0000         0
           6     0.1631    0.1631    0.1631       613

    accuracy                         0.1478      4291
   macro avg     0.1478    0.1095    0.1010      4291
weighted avg     0.3729    0.1478    0.1945      4291

Time taken to predict the model 25.431562185287476
